# Duas implementações do cripto esquema NTRU

# Parte A 
### NTRU Textbook

**parâmetros $N,t,p,q$**

Esta implementação é essencialmente "textbook" inicial mas tem algumas variantes seguindo as tendências recentes:
1. usa multiplicações sobre anéis ciclotómicos determinados por quotientes  $x^N+1$ sendo $N$ uma potência de $2$. 
1. o parâmetro $q$ não é da forma $2^d$ (para algum $d$) mas sim um primo da forma $1+p\,2^d$.

In [5]:
t = 64
p = ZZ(7)
N = 2**9              # N é uma potência de 2

d = 4
# definição de "q" como o menor primo tal que "(q-1)/p" é uma potência de "2" 
# maior ou igual a "2^d"
u = p << d
while not is_prime(u + 1):
    u = u << 1
q = u + 1

**Construção dos anéis de polinómios usados no NTRU**

In [6]:
Zx.<x> = PolynomialRing(ZZ)        # polinómios de coeficientes inteiros
Qx.<x> = PolynomialRing(GF(q))     # polinómios de coeficientes inteiros módulo "q"
Px.<x> = PolynomialRing(GF(p))     # polinómios de coeficientes inteiros módulo "p"

quo = (x^N+1)
# Versões dos polinómios anteriores módulo o polinómio ciclotómico
Zxr.<x> = Zx.quotient(Zx(quo))
Qxr.<x> = Qx.quotient(Qx(quo))
Pxr.<x> = Px.quotient(Px(quo))

**Funções auxiliares para fazer o 'lift' de  polinómios**

In [7]:
def _lift(pol):                      # converte qualquer polinómio em Zx
    return Zx(map(lift,pol.list()))

def _round(pol,r):             # análogo a _lift mas converte os coeficientes 
                               # ao intervalo simétrico  com r elementos (r é ímpar)
    u = map(lambda n: n%r, map(lift,pol.list())) 
    rr = r//2          
    return Zx([n if n <= rr else n - r  for n in u])



**Gerar  polinómios ternários aleatórios**

- `small_poly` : polinómios de coeficientes inteiros $\left\{-1,0,1\right\}$ em que o número de coeficientes não-zero é aproximadamente $t$ e, dentro destes, o número de $1$'s é aproximadamente igual ao número de $-1$'s.
- `poly`: polinómios de coeficientes inteiros $\left\{-1,0,1\right\}$ com todos os valores escolhidos com igual probabilidade

In [8]:
from random import *

# generators for  random polynomials with coefficients    [1,0,-1]

def small_poly(N):      # "small" significa que o número de elementos diferentes 
                       # de zero é aproximadamente t
    u = floor(2*N/t)
    v = 0 ; l = [0]*N 
    while True:
            v += randint(1,u)
            if v >= N-1:
                break
            else:
                l[v] = choice([-1,1])
    return Zx(l)
    
def poly(N):
    return  Zx([choice([-1,0,1]) for k in range(N)])

### A classe NTRUtxb

In [9]:
class NTRUtxb:
    def __init__(self):
        while True:
            f = small_poly(N)
            if Qxr(f).is_unit() and Pxr(f).is_unit():
                break
        fp = Pxr(f)^(-1) ; fq = Qxr(f)^(-1) ; w = small_poly(N)
        self.h = _lift(Qxr(w)*fq)
        self.pk = (f , _lift(fp))
        
    def encrypt(self,m):
        gamma = small_poly(N)
        return _lift(Qxr(m) + Qxr(p*gamma)*Qxr(self.h))

    def decrypt(self,e):
        (f,fp) = self.pk
        a = _round(Qxr(f)*Qxr(e),q)
        return  _round(Pxr(fp)*Pxr(a),p)

**Exemplo e Teste**

In [10]:
# Uma instância NTRU
K = NTRUtxb() 
# Uma mensagem aleatória
m = poly(N)
# Cifrar
e = K.encrypt(m)  
# Decifrar e verificar a correcção
m == K.decrypt(e)

False

**Questão**
Em termos de tamanho como se compara a mensagem $m$ com o criptograma $e$ ?

# Parte B

## NTRU Prime 
segundo o paper **NTRU Prime** de *D.J. Bernstein, C. Chuengsatiansup, T. Lange1 and C. van Vredendaal*.

**Parâmetros e geração dos anéis de polinómios**

In [15]:
t = 64
q = 24*t
while True:
    if (1+q).is_prime():
        break
    else:
        q += 3
q += 1
 
Zx.<x>  = ZZ[]
Z3.<y>  = PolynomialRing(GF(3))
Gq.<z>  = GF(q)[]
 
p = next_prime(2*t)
while True:
    if  Gq(x^p-x-1).is_irreducible():
        break
    else:
        p = next_prime(p+1)

Zxr.<x> = Zx.quotient(x^p-x-1)
Z3r.<y> = Z3.quotient(y^p-y-1)
Gqr.<z> = Gq.quotient(z^p-z-1)

# lifting

def _lift(w):
    return Zx(map(lift,w.list()))

In [16]:
# Teste de "casting"
# i.e. converter um polinómio do anel mais geral para um anel mais específico
u = Zx([1,0,0,1,3400000003,10000000031]) ; a = Z3r(u) ; b = Gqr(u)
print a ; print b

2*y^5 + y^4 + y^3 + 1
648*z^5 + 1046*z^4 + z^3 + 1


###  Funções auxiliares

In [17]:
# random polynomial: generadores, o operador weight e o "arredondamento"

from random import choice, randint

def _small_poly(p,t=None):
    if not t:
        return Zx([choice([-1,0,1]) for k in range(p)])
    u = floor(2*(p-1)//t) ; k = randint(0,u) ; l = [0]*p
    while k < p:
        l[k] = choice([-1,1]) ; k += randint(1,u)
    return Zx(l)

def _is_small(w):
    return reduce(lambda x,y : x and y^2 <= 1,w.list(),True)
    
def _weight(w):
    return reduce(lambda x,y: x+1 if y!=0 else x,w.list()) 
    
def _round(w,n=q):          
    """
         input:  polinómio em Gqr ou Z3r
         output: transpõe os coeficientes para o intervalo -n//2..+n//2
    """
    r = n//2
    return Zx(map(lambda x: lift(x + r) - r , w.list()))
    
def _round_3(w):        
    """ 
         transpõe os coeficientes de "w" para o intervalo -q//2..+q//2 
         e arredonda-os ao múltiplo de 3 mais próximo
    """
    def _f(x):
        return ((x/3).round())*3 
    r = q//2
    return  Zx(map(lambda x: _f(lift(x+r) - r) , w.list()))
    
import hashlib

def Hash(w):
    ww = reduce(lambda x,y: x + y.binary(), w.list() , "") 
    return hashlib.sha256(ww).hexdigest()

**Teste e verificação**

In [18]:
#u = Gqr(_small_poly(p)) * Gqr(_small_poly(p)) 
#_round_3(u)

## A classe NTRUprime

Implementada como um KEM

In [19]:
class NTRUprime(object):
    def __init__(self):
        g = _small_poly(p)
        while not Z3r(g).is_unit():
            g = _small_poly(p)
        f = _small_poly(p,t) ; g_inv = Z3r(g)^(-1)
        self.secret = (f , g_inv)                      # chave privada é um par (Zx, Z3r)
        self.pk = Gqr(g)/Gqr(3*f)                      # chave pública em Gqr
        
    def encapsulate(self):
        w = _small_poly(p,t)
        key = Hash(w)
        C   = _round_3(Gqr(w)*self.pk)
        return (key, C)
 
    def reveal(self,C):
        (f , s) = self.secret
        e = s * Z3r(_round(Gqr(3*f) * Gqr(C))) ; w = _round(e,n=3) ; 
        key = Hash(w)
        return key

** Teste e Verificação**

In [20]:
# instância e geração de chaves
K = NTRUprime()
# Cifrar
(key,C) = K.encapsulate() 
# Decifrar e verificar
key == K.reveal(C)

True